##Healthcare Stroke Patients Prediction - Pyspark

Team members - Abhijeet Ray, Deepak Rao , AKhil Menon, Kushagra Sen

## Objective 
* To forecast whether a patient can have stroke or not using the given information of patients. It is a classification problem, where we will try to predict the probability of an observation belonging to a category (in our case probability of having a stroke)

* Here we have clinical measurements (e.g. Hypertension, heart_disease, age, smoking_status) for a number of patients, as well as information about whether each patient has had a stroke. In practice, we are developing our model to accurately predict stroke risk for future patients based on their clinical measurements.

### Understand stroke
#### What is a stroke

* A stroke is a “brain attack”. It can happen to anyone at any time. It occurs when blood flow to an area of brain is cut off. When this happens, brain cells are deprived of oxygen and begin to die. When brain cells die during a stroke, abilities controlled by that area of the brain such as memory and muscle control are lost.

#### Stroke are the world’s biggest killers

* 800,000 strokes per year in the US
* Fitfh leading cause of death in the US
* Leading cause of adult disability in the US
* 80% are preventable

#### Understanding stroke attributes in the dataset

* This dataset contains clinical measurements of 43401 patients. Description of this dataset can be viewed on the Kaggle website, where the data was obtained (https://www.kaggle.com/asaumya/healthcare-dataset-stroke-data).

##### Data Description
* ID: Patient's ID, probably irrelevant unless to avoid duplicates

* GENDER - SEX (male ; female)

* AGE - age in years

* HYPERTENSION: (0- No ; 1- Yes) Hypertension is another name for high blood pressure, Can lead to severe complications and increases the risk of heart disease, stroke, and death. Normal blood pressure is 120 over 80 mm of mercury (mmHg), but hypertension is higher than 130 over 80 mmHg. (Source: https://www.medicalnewstoday.com/articles/150109.php)

* HEART_DISEASE: (0- No ; 1- Yes), could cause increase stroke risks.
     
* EVER_MARRIED: (YES or NO) Lifestyle factor. Is this relevant? -We will need to test.

* TYPE_OF_WORK: Did the person worked as a Government servant or in a Private organization or was Self employed? Lifestyle factor. Is this relevant? -We will need to test.

* RESIDENCE: Home location (Rural or Urban), Lifestyle factor. Is this relevant? -We will need to test.

* AVG_GLUCOSE: Average Glucose level in the person, could be relevant.

* BMI: Body Mass Index. Could be relevant, to test.

     *An index for assessing overweight and underweight, obtained by dividing body weight in kg by height in m^2
     
     *A measure of 25 or more is considered overweight.
     
* Smoking_Status : Smoking habbits (Smoking, Formerly smoked, occassional smoker), its a Lifestyle factor.

##### Predictor Field
* Stroke: Did the person had stroke ? (0-No ; 1-Yes)

### Importing necessary modules

In [6]:
from pyspark.sql import SparkSession
import pyspark.sql as sparksql
import pandas as pd
import numpy as np
#Data visualisation libraries 
import matplotlib.pyplot as plt   
import seaborn as sns
#importing ml features
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline,Model
from pyspark.ml.feature import StringIndexer

In [7]:
%sh
wget -O train https://www.dropbox.com/s/p3sy3sxj71gxjtx/train_2v.csv?dl=0

--2020-01-18 18:19:35-- https://www.dropbox.com/s/p3sy3sxj71gxjtx/train_2v.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/p3sy3sxj71gxjtx/train_2v.csv [following]
--2020-01-18 18:19:36-- https://www.dropbox.com/s/raw/p3sy3sxj71gxjtx/train_2v.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8eef496296900a67cb97f7aa1e.dl.dropboxusercontent.com/cd/0/inline/AwZyLSfiS54CYN-qU6OEUXuprABgsutlKkQkNrYy_qus9-eIVjHcn5tYz361Kz28IMda4vU9EqdTNcxRkONVJoNAqi2VgZLEi4b8wg-5ULot3DiYRz86ruQ8j88MOeEST08/file# [following]
--2020-01-18 18:19:36-- https://uc8eef496296900a67cb97f7aa1e.dl.dropboxusercontent.com/cd/0/inline/AwZyLSfiS54CYN-qU6OEUXuprABgsutlKkQkNrYy_qus9-eIVjHcn5tYz361Kz28IMda4vU9EqdTNcxRkONVJoNAqi2VgZLEi4b8wg-5ULot3DiYRz86ruQ8j88MOeEST08/file
Resolving uc8eef496296900a67cb97f7aa1e.dl.dropboxusercontent.com (uc8eef496296900a67cb97f7aa1e.dl.dropboxusercontent.com)... 162.125.1.6, 2620:100:6016:6::a27d:106
Connecting to uc8eef496296900a67cb97f7aa1e.dl.dropboxusercontent.com (uc8eef496296900a67cb97f7aa1e.dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2635787 (2.5M) [text/plain]
Saving to: ‘train’

 0K .......... .......... .......... .......... .......... 1% 3.16M 1s
 50K .......... .......... .......... .......... .......... 3% 10.9M 0s
 100K .......... .......... .......... .......... .......... 5% 10.7M 0s
 150K .......... .......... .......... .......... .......... 7% 15.1M 0s
 200K .......... .......... .......... .......... .......... 9% 18.9M 0s
 250K .......... .......... .......... .......... .......... 11% 36.8M 0s
 300K .......... .......... .......... .......... .......... 13% 25.5M 0s
 350K .......... .......... .......... .......... .......... 15% 20.6M 0s
 400K .......... .......... .......... .......... .......... 17% 45.0M 0s
 450K .......... .......... .......... .......... .......... 19% 30.0M 0s
 500K .......... .......... .......... .......... .......... 21% 34.6M 0s
 550K .......... .......... .......... .......... .......... 23% 99.1M 0s
 600K .......... .......... .......... .......... .......... 25% 55.9M 0s
 650K .......... .......... .......... .......... .......... 27% 62.7M 0s
 700K .......... .......... .......... .......... .......... 29% 65.3M 0s
 750K .......... .......... .......... .......... .......... 31% 58.4M 0s
 800K .......... .......... .......... .......... .......... 33% 69.4M 0s
 850K .......... .......... .......... .......... .......... 34% 42.9M 0s
 900K .......... .......... .......... .......... .......... 36% 63.4M 0s
 950K .......... .......... .......... .......... .......... 38% 117M 0s
 1000K .......... .......... .......... .......... .......... 40% 96.8M 0s
 1050K .......... .......... .......... .......... .......... 42% 70.0M 0s
 1100K .......... .......... .......... .......... .......... 44% 140M 0s
 1150K .......... .......... .......... .......... .......... 46% 62.6M 0s
 1200K .......... .......... .......... .......... .......... 48% 114M 0s
 1250K .......... .......... .......... .......... .......... 50% 139M 0s
 1300K .......... .......... .......... .......... .......... 52% 120M 0s
 1350K .......... .......... .......... .......... .......... 54% 129M 0s
 1400K .......... .......... .......... .......... .......... 56% 140M 0s
 1450K .......... .......... .......... .......... .......... 58% 132M 0s
 1500K .......... .......... .......... .......... .......... 60% 126M 0s
 1550K .......... .......... .......... .......... .......... 62% 112M 0s
 1600K .......... .......... .......... .......... .......... 64% 138M 0s
 1650K .......... .......... .......... .......... .......... 66% 123M 0s
 1700K .......... .......... .......... .

### Loading the dataset

In [9]:
train = spark.read.csv(path='file:///databricks/driver/train', inferSchema=True,header=True)

### Exploring the data

In [11]:
train.dtypes

Out[4]: [('id', 'int'),
 ('gender', 'string'),
 ('age', 'double'),
 ('hypertension', 'int'),
 ('heart_disease', 'int'),
 ('ever_married', 'string'),
 ('work_type', 'string'),
 ('Residence_type', 'string'),
 ('avg_glucose_level', 'double'),
 ('bmi', 'double'),
 ('smoking_status', 'string'),
 ('stroke', 'int')]

In [12]:
train.columns

Out[5]: ['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke']

In [13]:
train.printSchema()

root
-- id: integer (nullable = true)
-- gender: string (nullable = true)
-- age: double (nullable = true)
-- hypertension: integer (nullable = true)
-- heart_disease: integer (nullable = true)
-- ever_married: string (nullable = true)
-- work_type: string (nullable = true)
-- Residence_type: string (nullable = true)
-- avg_glucose_level: double (nullable = true)
-- bmi: double (nullable = true)
-- smoking_status: string (nullable = true)
-- stroke: integer (nullable = true)

In [14]:
train.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
 id|gender| age|hypertension|heart_disease|ever_married| work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
30669| Male| 3.0| 0| 0| No| children| Rural| 95.12|18.0| null| 0|
30468| Male|58.0| 1| 0| Yes| Private| Urban| 87.96|39.2| never smoked| 0|
16523|Female| 8.0| 0| 0| No| Private| Urban| 110.89|17.6| null| 0|
56543|Female|70.0| 0| 0| Yes| Private| Rural| 69.04|35.9|formerly smoked| 0|
46136| Male|14.0| 0| 0| No| Never_worked| Rural| 161.28|19.1| null| 0|
32257|Female|47.0| 0| 0| Yes| Private| Urban| 210.95|50.1| null| 0|
52800|Female|52.0| 0| 0| Yes| Private| Urban| 77.59|17.7|formerly smoked| 0|
41413|Female|75.0| 0| 1| Yes|Self-employed| Rural| 243.53|27.0| never smoked| 0|
15266|Female|32.0| 0| 0| Yes| Private| Rural| 77.67|32.3| smokes| 0|
28674|Female|74.0| 1| 0| Yes|Self-employed| Urban| 205.84|54.6| never smoked| 0|
10460|Female|79.0| 0| 0| Yes| Govt_job| Urban| 77.08|35.0| null| 0|
64908| Male|79.0| 0| 1| Yes| Private| Urban| 57.08|22.0|formerly smoked| 0|
63884|Female|37.0| 0| 0| Yes| Private| Rural| 162.96|39.4| never smoked| 0|
37893|Female|37.0| 0| 0| Yes| Private| Rural| 73.5|26.1|formerly smoked| 0|
67855|Female|40.0| 0| 0| Yes| Private| Rural| 95.04|42.4| never smoked| 0|
25774| Male|35.0| 0| 0| No| Private| Rural| 85.37|33.0| never smoked| 0|
19584|Female|20.0| 0| 0| No| Private| Urban| 84.62|19.7| smokes| 0|
24447|Female|42.0| 0| 0| Yes| Private| Rural| 82.67|22.5| never smoked| 0|
49589|Female|44.0| 0| 0| Yes| Govt_job| Urban| 57.33|24.6| smokes| 0|
17986|Female|79.0| 0| 1| Yes|Self-employed| Urban| 67.84|25.2| smokes| 0|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
only showing top 20 rows

### Visualizing the Data

In [16]:
pdata = train.toPandas()
g = sns.pairplot(pdata, size=1.2)
display(g.fig)

In [17]:
fig, ax = plt.subplots()
fig.set_size_inches(7, 5)
sns.heatmap(pdata.corr(),annot=True ,cmap='magma').set_title('Correlation Factors Heat Map', color='black', size='20')
display(fig)

* It is clearly evident that Age and heart disease are the factors that are highly correlated with Stroke, followed by avg_glucose_level, hypertension and then bmi

#### Training feature analysis

In [20]:
# create DataFrame as a temporary view for SQL queries
train.createOrReplaceTempView('table')

#### Work type vs stroke (influence of work type on getting stroke)

In [22]:
# sql query to find the number of people in specific work_type who have had stroke and not
spark.sql("SELECT work_type, COUNT(work_type) as work_type_count \
          FROM table WHERE stroke == 1 \
          GROUP BY work_type \
          ORDER BY COUNT(work_type) DESC").show()

+-------------+---------------+
 work_type|work_type_count|
+-------------+---------------+
 Private| 441|
Self-employed| 251|
 Govt_job| 89|
 children| 2|
+-------------+---------------+

* The most affected work_type persons are private followed by self-employed.

#### Gender vs Stroke (is strokes related to gender ? )

In [25]:
spark.sql("SELECT gender, COUNT(gender) as gender_count, COUNT(gender)*100/(SELECT COUNT(gender) FROM table WHERE gender == 'Male') as    percentage \
          FROM table WHERE stroke== 1 AND gender = 'Male' \
          GROUP BY gender").show()
spark.sql("SELECT gender, COUNT(gender) as gender_count, COUNT(gender)*100/(SELECT COUNT(gender) FROM table WHERE gender == 'Female') as percentage \
          FROM table WHERE stroke== 1 AND gender = 'Female' \
          GROUP BY gender").show()

+------+------------+------------------+
gender|gender_count| percentage|
+------+------------+------------------+
 Male| 352|1.9860076732114647|
+------+------------+------------------+

+------+------------+------------------+
gender|gender_count| percentage|
+------+------------+------------------+
Female| 431|1.6793298266121177|
+------+------------+------------------+

* 1.68% male and almost 2% male had stroke.

#### Age vs Stroke (Now we will see influence of age on stroke)

In [28]:
spark.sql("SELECT COUNT(age)*100/(SELECT COUNT(age) FROM table WHERE stroke ==1) as percentage \
          FROM table \
          WHERE stroke == 1 AND age<50").show()
spark.sql("SELECT COUNT(age)*100/(SELECT COUNT(age) FROM table WHERE stroke ==1) as percentage \
          FROM table \
          WHERE stroke == 1 AND age>=50").show()

+----------------+
 percentage|
+----------------+
8.42911877394636|
+----------------+

+-----------------+
 percentage|
+-----------------+
91.57088122605364|
+-----------------+

* It is clearly evident that older age have a strong correlation with Strokes, as 91.5% stroke had occured for people who are more than 50 years old and only 8% stroke for people whose age is less than 50

### Cleaning up training data

In [31]:
train.describe().show()

+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+---------------+-------------------+
summary| id|gender| age| hypertension| heart_disease|ever_married|work_type|Residence_type| avg_glucose_level| bmi| smoking_status| stroke|
+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+---------------+-------------------+
 count| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 41938| 30108| 43400|
 mean|36326.14235023042| null| 42.21789400921646|0.09357142857142857|0.04751152073732719| null| null| null|104.48274999999916|28.605038390004545| null|0.01804147465437788|
 stddev|21072.13487918279| null|22.519648680503554| 0.291234906309397|0.21273274050209726| null| null| null| 43.1117509512961| 7.770020497238766| null|0.13310292280179215|
 min| 1|Female| 0.08| 0| 0| No| Govt_job| Rural| 55.0| 10.1|formerly smoked| 0|
 max| 72943| Other| 82.0| 1| 1| Yes| children| Urban| 291.05| 97.6| smokes| 1|
+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+---------------+-------------------+

d
 * Here we see that the count for each of the columns is 43400 except "smoking status" and "bmi". This indicates there are few missing values in "smoking_status" and "bmi"
 * Also there are few categorical data (gender, ever_married, work_type, Residence_type, smoking_status) which we need to covert using one hot encoding

In [33]:
# fill in missing values for column "smoking status"
# As "smoking status" column is categorical data, we will add one data type "No Info" for the missing one

train_f = train.na.fill('No Info', subset=['smoking_status'])

In [34]:
# fill in missing values for column "bmi" 
# as column "bmi" is numerical data , we will fill the missing values with mean

from pyspark.sql.functions import mean
mean = train_f.select(mean(train_f['bmi'])).collect()
mean_bmi = mean[0][0]
train_f = train_f.na.fill(mean_bmi,['bmi'])

In [35]:
train_f.describe().show()

+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+--------------+-------------------+
summary| id|gender| age| hypertension| heart_disease|ever_married|work_type|Residence_type| avg_glucose_level| bmi|smoking_status| stroke|
+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+--------------+-------------------+
 count| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 43400| 43400|
 mean|36326.14235023042| null| 42.21789400921646|0.09357142857142857|0.04751152073732719| null| null| null|104.48274999999916|28.605038390005145| null|0.01804147465437788|
 stddev|21072.13487918279| null|22.519648680503554| 0.291234906309397|0.21273274050209726| null| null| null| 43.1117509512961| 7.638023372051845| null|0.13310292280179215|
 min| 1|Female| 0.08| 0| 0| No| Govt_job| Rural| 55.0| 10.1| No Info| 0|
 max| 72943| Other| 82.0| 1| 1| Yes| children| Urban| 291.05| 97.6| smokes| 1|
+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+--------------+-------------------+

d
 * Here we see that the count for each of the columns now is 43400. This indicates the missing values in "smoking_status" and "bmi" has been handled

### Data Modelling

* Now, Lets work on categorical columns.
* Perfroming "StringIndexer -> OneHotEncoder -> VectorAssembler"

In [39]:
# indexing all categorical columns in the dataset

indexer1 = StringIndexer(inputCol="gender", outputCol="genderIndex")
indexer2 = StringIndexer(inputCol="ever_married", outputCol="ever_marriedIndex")
indexer3 = StringIndexer(inputCol="work_type", outputCol="work_typeIndex")
indexer4 = StringIndexer(inputCol="Residence_type", outputCol="Residence_typeIndex")
indexer5 = StringIndexer(inputCol="smoking_status", outputCol="smoking_statusIndex")

In [40]:
# Doing one hot encoding of indexed data

from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["genderIndex","ever_marriedIndex","work_typeIndex","Residence_typeIndex","smoking_statusIndex"],
                                 outputCols=["genderVec","ever_marriedVec","work_typeVec","Residence_typeVec","smoking_statusVec"])

In [41]:
#The next step is to create an assembler, that combines a given list of columns into a single vector column to train ML model. We will use the vector columns, that we got after one_hot_encoding.
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[
 'genderVec',
 'age',
 'hypertension',
 'heart_disease',
 'ever_marriedVec',
 'work_typeVec',
 'Residence_typeVec',
 'avg_glucose_level',
 'bmi',
 'smoking_statusVec'] , outputCol='features')

* We have complex task that contains bunch of stages, these bunch of satges needs to be performed to process data. To wrap all of that Spark ML represents such a workflow as a Pipeline, which consists of a sequence of PipelineStages to be run in a specific order.

In [43]:
from pyspark.ml import Pipeline
# Pipeline basic to be shared across model fitting and testing
pipeline = Pipeline(stages=[]) # Must initialize with empty list!

In [44]:
basePipeline = [indexer1, indexer2, indexer3, indexer4, indexer5, encoder, assembler]

In [45]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, NaiveBayes, LogisticRegression, DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

dtc = DecisionTreeClassifier(labelCol='stroke',featuresCol='features')
pl_dtc=basePipeline+[dtc]
pg_dtc = ParamGridBuilder()\
        .baseOn({pipeline.stages: pl_dtc})\
        .build()

rf = RandomForestClassifier(featuresCol="features", labelCol="stroke",numTrees=25)
pl_rf = basePipeline + [rf]
pg_rf = ParamGridBuilder()\
      .baseOn({pipeline.stages: pl_rf})\
      .build()

nb = NaiveBayes(labelCol='stroke',featuresCol='features')
pl_nb = basePipeline + [nb]
pg_nb = ParamGridBuilder()\
.baseOn({pipeline.stages: pl_nb})\
.addGrid(nb.smoothing,[0.4,1.0])\
.build()

paramGrid = pg_dtc + pg_rf + pg_nb


###Data Transformation

In [47]:
splitted_data = train_f.randomSplit([0.7,0.3], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 30379
Number of testing records : 13021

In [48]:
cv = CrossValidator()\
    .setEstimator(pipeline)\
    .setEvaluator(MulticlassClassificationEvaluator(labelCol="stroke", predictionCol="prediction", metricName="accuracy"))\
    .setEstimatorParamMaps(paramGrid)\
    .setNumFolds(3) 

In [49]:
fittedModel1=cv.fit(train_data)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [50]:
train_data.show()

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+------------------+---------------+------+
 id|gender| age|hypertension|heart_disease|ever_married| work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+------------------+---------------+------+
 6|Female|21.0| 0| 0| No| Private| Urban| 75.73|28.605038390004545| never smoked| 0|
 8|Female|79.0| 0| 0| Yes| Private| Urban| 99.23| 25.1| No Info| 0|
 11|Female|54.0| 0| 0| Yes|Self-employed| Urban| 83.01|28.605038390004545| never smoked| 0|
 12|Female|56.0| 0| 0| Yes| Private| Rural| 102.3| 54.9|formerly smoked| 0|
 14|Female|67.0| 0| 0| Yes| Private| Urban| 111.04| 25.1| never smoked| 0|
 15|Female|74.0| 0| 0| No| Govt_job| Rural| 104.73|28.605038390004545| No Info| 0|
 22| Male| 5.0| 0| 0| No| children| Rural| 100.81| 20.6| No Info| 0|
 25|Female| 4.0| 0| 0| No| children| Urban| 71.9| 14.1| No Info| 0|
 36| Male| 5.0| 0| 0| No| children| Rural| 91.85| 16.6| No Info| 0|
 37|Female|13.0| 0| 0| No| Private| Rural| 83.99| 22.5| No Info| 0|
 39| Male|52.0| 1| 0| Yes|Self-employed| Rural| 200.37|28.605038390004545| smokes| 0|
 40|Female|36.0| 0| 0| No| Private| Rural| 65.85| 59.0| never smoked| 0|
 41| Male|74.0| 0| 0| Yes| Govt_job| Rural| 201.58| 25.1|formerly smoked| 0|
 46|Female|46.0| 0| 0| Yes| Govt_job| Urban| 80.97| 25.4| never smoked| 0|
 47|Female|28.0| 0| 0| Yes| Private| Urban| 94.49| 20.0| never smoked| 0|
 48|Female|58.0| 0| 0| Yes| Govt_job| Rural| 77.54| 18.1| never smoked| 0|
 51| Male|59.0| 0| 0| Yes| Govt_job| Urban| 91.02|28.605038390004545| never smoked| 0|
 54|Female|52.0| 0| 0| Yes|Self-employed| Urban| 199.55|28.605038390004545|formerly smoked| 0|
 56| Male|48.0| 0| 0| Yes|Self-employed| Rural| 201.55| 39.7|formerly smoked| 0|
 65|Female|26.0| 0| 0| Yes| Private| Rural| 119.28| 32.4|formerly smoked| 0|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+------------------+---------------+------+
only showing top 20 rows

###The Best Model

In [52]:
import numpy as np
# BinaryClassificationEvaluator defaults to ROC AUC, so higher is better
# http://gim.unmc.edu/dxtests/roc3.htm
fittedModel1.getEstimatorParamMaps()[ np.argmax(fittedModel1.avgMetrics) ]

Out[28]: {Param(parent='Pipeline_699e189f94b6', name='stages', doc='a list of pipeline stages'): [StringIndexer_2f2fdfff9653,
 StringIndexer_e979ef558108,
 StringIndexer_65ef13cccb45,
 StringIndexer_7d85213125a3,
 StringIndexer_50a2ee2c5949,
 OneHotEncoderEstimator_1139ed206783,
 VectorAssembler_06016f2f5c8d,
 RandomForestClassifier_a7c0f599b9c6]}

* As is clearly evident, RandomeForestCLassifier is coming out to be the best model

###The Worst Model

In [55]:
fittedModel1.getEstimatorParamMaps()[ np.argmin(fittedModel1.avgMetrics) ]

Out[29]: {Param(parent='Pipeline_699e189f94b6', name='stages', doc='a list of pipeline stages'): [StringIndexer_2f2fdfff9653,
 StringIndexer_e979ef558108,
 StringIndexer_65ef13cccb45,
 StringIndexer_7d85213125a3,
 StringIndexer_50a2ee2c5949,
 OneHotEncoderEstimator_1139ed206783,
 VectorAssembler_06016f2f5c8d,
 NaiveBayes_70c961cbfbc1],
 Param(parent='NaiveBayes_70c961cbfbc1', name='smoothing', doc='The smoothing parameter, should be >= 0, default is 1.0'): 1.0}

* NaiveBayes is worst model

In [57]:
import re
def paramGrid_model_name(model):
  params = [v for v in model.values() if type(v) is not list]
  name = [v[-1] for v in model.values() if type(v) is list][0]
  name = re.match(r'([a-zA-Z]*)', str(name)).groups()[0]
  return "{}{}".format(name,params)

# Resulting metric and model description
# get the measure from the CrossValidator, cvModel.avgMetrics
# get the model name & params from the paramGrid
# put them together here:
kmeans_measures = zip(fittedModel1.avgMetrics, [paramGrid_model_name(m) for m in paramGrid])
metrics,model_names = zip(*kmeans_measures)

In [58]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.clf() # clear figure
fig = plt.figure( figsize=(5, 5))
plt.style.use('fivethirtyeight')
axis = fig.add_axes([0.1, 0.3, 0.8, 0.6])
# plot the metrics as Y
#plt.plot(range(len(model_names)),metrics)
plt.bar(range(len(model_names)),metrics)
# plot the model name & param as X labels
plt.xticks(range(len(model_names)), model_names, rotation=70, fontsize=6)
plt.yticks(fontsize=6)
#plt.xlabel('model',fontsize=8)
plt.ylabel('ROC AUC (better is greater)',fontsize=8)
plt.title('Model evaluations')
display(plt.show())

### Predictions

In [60]:
predictions=fittedModel1.transform(test_data)

In [61]:
## Make predictions on test documents. 
# CrossValidator.fit() is in cvModel, which is the best model found (rfModel).
predictions.select("prediction", "stroke").show(5)

+----------+------+
prediction|stroke|
+----------+------+
 0.0| 0|
 0.0| 0|
 0.0| 0|
 0.0| 0|
 0.0| 0|
+----------+------+
only showing top 5 rows

### Model Evaluation

In [63]:
# Select (prediction, true label) and compute test error

acc_evaluator = MulticlassClassificationEvaluator(labelCol="stroke", predictionCol="prediction", metricName="accuracy")
rfc_acc = acc_evaluator.evaluate(predictions)
print('A Random Forest Classifier had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))

A Random Forest Classifier had an accuracy of: 98.30%

#### So, the best model which is the RandomForestClassifier gives us the accuracy of 98.30% which is very high, so our model's accuracy seems to be great

####Lets view result of the best model

In [66]:
correct = predictions.where("(stroke = prediction)").count()
incorrect = predictions.where("(stroke != prediction)").count()

resultDF = sqlContext.createDataFrame([['correct', correct], ['incorrect', incorrect]], ['metric', 'value'])
display(resultDF)

metric,value
correct,12800
incorrect,221


## Result Visualization

In [68]:
correct = predictions.where("(stroke = prediction)").count()
incorrect = predictions.where("(stroke != prediction)").count()

resultDF = sqlContext.createDataFrame([['correct', correct], ['incorrect', incorrect]], ['metric', 'value'])
display(resultDF)

metric,value
correct,12800
incorrect,221


#### As is evident from the above visualization, our model predicted very high correct classifications over incorrect classifications which clearly indicates model is performing good.

####Confusion Matrix

In [71]:
counts = [predictions.where('stroke=1').count(), predictions.where('prediction=1').count(),
          predictions.where('stroke=0').count(), predictions.where('prediction=0').count()]
names = ['actual 1', 'predicted 1', 'actual 0', 'predicted 0']
display(sqlContext.createDataFrame(zip(names,counts),['Measure','Value']))

Measure,Value
actual 1,221
predicted 1,0
actual 0,12800
predicted 0,13021


In [72]:
counts = [predictions.where('stroke=1').count(), predictions.where('prediction=1').count(),
          predictions.where('stroke=0').count(), predictions.where('prediction=0').count()]
names = ['actual 1', 'predicted 1', 'actual 0', 'predicted 0']
display(sqlContext.createDataFrame(zip(names,counts),['Measure','Value']))

Measure,Value
actual 1,221
predicted 1,0
actual 0,12800
predicted 0,13021
